In [5]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import os
import pandas as pd
from datetime import datetime
import os
import pandas as pd
from datetime import datetime

# Función para convertir coordenadas en formato E7 a valores decimales
def e7_to_decimal(e7_value):
    if e7_value is None:
        return None
    return e7_value * 1e-7

# Función para formatear coordenadas en formato decimal a español con comas
def format_coordinates(latitude, longitude):
    if latitude is None or longitude is None:
        return None
    return f"{latitude:.7f}, {longitude:.7f}"

# Función para convertir una cadena ISO8601 en un objeto datetime
def iso8601_to_datetime(iso8601_string):
    if iso8601_string is None:
        return None
    try:
        return datetime.strptime(iso8601_string, '%Y-%m-%dT%H:%M:%S.%fZ')
    except ValueError:
        return datetime.strptime(iso8601_string, '%Y-%m-%dT%H:%M:%SZ')

# Función para calcular la duración formateada
def format_duration(start_timestamp, end_timestamp):
    if start_timestamp is None or end_timestamp is None:
        return None
    
    # Calcular la diferencia en segundos
    time_difference = (end_timestamp - start_timestamp).total_seconds()
    
    # Calcular los componentes de la duración en días, horas, minutos y segundos
    days = int(time_difference // (24 * 3600))
    time_difference %= 24 * 3600
    hours = int(time_difference // 3600)
    time_difference %= 3600
    minutes = int(time_difference // 60)
    seconds = int(time_difference % 60)
    
    # Formatear la duración en el formato "DD:HH:MM:SS"
    duration_formatted = f"{days:02d}:{hours:02d}:{minutes:02d}:{seconds:02d}"
    return duration_formatted

# Función para extraer datos de cada celda de la columna 'activitySegment' y generar columnas
def extract_data(cell):
    activity = cell
    start_timestamp = iso8601_to_datetime(activity['duration'].get('startTimestamp'))
    end_timestamp = iso8601_to_datetime(activity['duration'].get('endTimestamp'))
    
    return pd.Series({
        'start_latitude': e7_to_decimal(activity['startLocation'].get('latitudeE7')),
        'start_longitude': e7_to_decimal(activity['startLocation'].get('longitudeE7')),
        'end_latitude': e7_to_decimal(activity['endLocation'].get('latitudeE7')),
        'end_longitude': e7_to_decimal(activity['endLocation'].get('longitudeE7')),
        'start_timestamp': start_timestamp,
        'end_timestamp': end_timestamp,
        'duration_formatted': format_duration(start_timestamp, end_timestamp),
        'distance': activity.get('distance'),
        'activity_type': activity.get('activityType'),
        'confidence': activity.get('confidence'),
        # Agrega aquí las columnas adicionales que quieras extraer de la fila
    }).fillna(value=pd.NA)

# Carpeta raíz que contiene las carpetas "2014" hasta "2023"
root_folder = '/Users/valentincortespuya/Downloads/Takeout/Historial de ubicaciones/Semantic Location History'

# Lista para almacenar todos los DataFrames resultantes
all_dfs = []

# Recorrer todas las carpetas desde "2022" hasta "2023"
for year in range(2021, 2024):
    year_folder = os.path.join(root_folder, str(year))
    if os.path.exists(year_folder):
        # Recorrer todas las carpetas y subcarpetas dentro de cada año
        for dirpath, dirnames, filenames in os.walk(year_folder):
            for filename in filenames:
                if filename.endswith('.json'):
                    # Leer el archivo JSON y cargarlo como un DataFrame
                    file_path = os.path.join(dirpath, filename)
                    df = pd.read_json(file_path)

                    # Extraer columnas 'timelineObjects' y expandir en un nuevo DataFrame
                    df2 = df['timelineObjects'].apply(pd.Series)

                    # Eliminar filas con NaN en la columna 'activitySegment'
                    df2_cleaned = df2.dropna(subset=['activitySegment'])

                    # Aplicar la función extract_data y expandir las columnas
                    df_extracted = df2_cleaned['activitySegment'].apply(extract_data)

                    # Concatenar las columnas extraídas con el DataFrame original
                    df_final = pd.concat([df2_cleaned, df_extracted], axis=1)

                    # Eliminar la columna original 'activitySegment' que contenía los datos completos (opcional)
                    df_final = df_final.drop(columns=['activitySegment'])

                    # Agregar el DataFrame resultante a la lista
                    all_dfs.append(df_final)

# Concatenar todos los DataFrames en uno solo
df_combined = pd.concat(all_dfs, ignore_index=True)

# Mostrar el DataFrame resultante
df_combined

# Exportar el DataFrame combinado a un archivo Excel
df_combined.to_csv("ubicaciones_historicas_usuario2.csv", index=False)

KeyError: ['activitySegment']

In [ ]:
!pip install folium

In [ ]:
import folium

# Crear un mapa centrado en una ubicación específica
map_center = [40.468159, -3.875562]
m = folium.Map(location=map_center, zoom_start=10)

# Iterar a través del DataFrame y agregar marcadores al mapa
for index, row in df_combined.iterrows():
    start_lat = row['start_latitude']
    start_lon = row['start_longitude']
    
    if not pd.isna(start_lat) and not pd.isna(start_lon):
        folium.Marker([start_lat, start_lon]).add_to(m)

# Guardar el mapa como un archivo HTML
m.save('mapa.html')


In [ ]:
import folium

# Crear un mapa centrado en una ubicación específica
map_center = [40.468159, -3.875562]
m = folium.Map(location=map_center, zoom_start=10)

# Iterar a través del DataFrame y agregar marcadores al mapa
for index, row in df_combined.iterrows():
    start_lat = row['start_latitude']
    start_lon = row['start_longitude']
    end_lat = row['end_latitude']
    end_lon = row['end_longitude']
    
    if not pd.isna(start_lat) and not pd.isna(start_lon) and not pd.isna(end_lat) and not pd.isna(end_lon):
        folium.Marker([start_lat, start_lon]).add_to(m)
        folium.Marker([end_lat, end_lon]).add_to(m)
        folium.PolyLine([(start_lat, start_lon), (end_lat, end_lon)], color="blue").add_to(m)

# Guardar el mapa como un archivo HTML
m.save('mapa_con_lineas_filtro-trasnporte.html')


In [ ]:
df_combined


In [ ]:
!pip install calplot

In [ ]:


import numpy as np
import pandas as pd
import calplot
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame df_combined con columnas 'start_timestamp' y 'activity_type'
# Reemplaza 'df_combined' con el nombre real de tu DataFrame

# Convierte la columna 'start_timestamp' en tipo datetime si aún no lo está
df_combined['start_timestamp'] = pd.to_datetime(df_combined['start_timestamp'])

# Configura el índice como un DatetimeIndex
actividades_por_dia = df_combined.groupby(df_combined['start_timestamp'].dt.floor('D'))['activity_type'].count()

# Crea el gráfico de calendario
calplot.calplot(actividades_por_dia,
                suptitle='Calendario',
                suptitle_kws={'x': 0.0, 'y': 1.0})

# Muestra el gráfico
plt.show()

